# Ejercicio de repaso: BBDD Relacionales

Ejercicio sacado directamente de una entrevista de trabajo para una conocida compañía de retail.

![tablas1](../../../img/Tablas-1.png)
![tablas3](../../../img/Tablas-3.png)

In [1]:
import numpy as np
import pandas as pd
import sqlite3
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("ECI.db")
# Obtenemos un cursor que utilizaremos para hacer las queries
crsr = connection.cursor()

# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)



## 1. Crea un DataFrame por cada tabla

In [2]:
dict_DEPTO_CAMPANIA = {'ID_DPTO':['1','1','2', '3', '3', '3', '4', '4', '4', '4'],
                'ID_CAMPANIA':['CA1', 'CA4', 'CA2', 'CA2', 'CA5', 'CA6', 'CA1', 'CA2', 'CA3', 'CA6']}
DEPTO_CAMPANIA = pd.DataFrame(dict_DEPTO_CAMPANIA)

DATA_DEPARTAMENTOS = {
        'ID_DPTO': ['1', '2', '3', '4'],
        'DESC_DPTO': ['JOYERIA', 'BOLSOS', 'ZAPATERIA', 'BAÑO CAB.']}
DEPARTAMENTO = pd.DataFrame(DATA_DEPARTAMENTOS)

familia = [['1', '1', '2', '3', '3', '4'], 
           ['1F1', '1F2', '2F1', '3F1', '3F2', '4F1'], 
           ['PULSERAS', 'ANILLOS', 'BOLSOS DE MANO', 'NAUTICO', 'SANDALIAS', 'BERMUDAS']]

columnas = ['ID_DPTO', 'COD_FAMILIA', 'DESC_FAMILIA']
FAMILIA = pd.DataFrame(dict(zip(columnas, familia)))

dic_campanias =  { "ID_CAMPANIA" : ["CA1", "CA2", "CA3", "CA4", "CA5", "CA6"], "TIPO": ["VENTA", "VENTA", "POSVENTA", "VENTA", "POSVENTA", "POSVENTA"], "FECHA_CAMP" : ["05/02/2012", "04/02/2012", "02/10/2009", "06/05/2010", "30/11/2010", "07/02/2015"] }
CAMPANIA = pd.DataFrame(dic_campanias)

Referencias = pd.Series(['A','B','C','D','E','F','G','H'])
ID_DPTO = pd.Series(['1','2','2','2','2','3','3','4'])
COD_FAMILIA =pd.Series(['1F1','2F1','2F1','2F1','2F1','3F1','3F2','4F1'])
FECHA_ALTA = pd.Series(['21/02/2010','07/02/2014','15/05/2014','01/10/2005','31/10/2010','20/04/2001','14/02/2015','25/03/2009'])
ARTICULO = pd.DataFrame({'REFERENCIA': Referencias,'ID_DPTO': ID_DPTO,'COD_FAMILIA':COD_FAMILIA,'FECHA_ALTA':FECHA_ALTA})

talon = ['01', '02', '03', '04', '05', '06', '07', '08']
referencia = ['A', 'A', 'A', 'D', 'E', 'E', 'H', 'H']
precio = [15, 18, 14, 37.95, 125.90, 150, 22.99, 24]
fecha_venta = ['11/02/2012', '15/02/2012', '02/10/2013', '06/08/2010', '30/11/2010', '05/02/2011', '11/04/2011', '08/08/2011']
VENTA = pd.DataFrame({'TALON':talon,
                    'REFERENCIA':referencia,
                    'PRECIO':precio,
                    'FECHA_VENTA':fecha_venta})

In [3]:
DEPTO_CAMPANIA.to_sql('DEPTO_CAMPANIA', connection, if_exists='replace', index=False)
DEPARTAMENTO.to_sql('DEPARTAMENTO', connection, if_exists='replace', index=False)
FAMILIA.to_sql('FAMILIA', connection, if_exists='replace', index=False)
CAMPANIA.to_sql('CAMPANIA', connection, if_exists='replace', index=False)
ARTICULO.to_sql('ARTICULO', connection, if_exists='replace', index=False)
VENTA.to_sql('VENTA', connection, if_exists='replace', index=False)
connection.commit()

## 2. Genera una consulta que obtenga la lista ordenada de todas las referencias sin venta

## 3. Genera una consulta que obtenga las ventas comprendidas entre 2010 y 2011 (ambos incluidos), que estén asociados a una campaña de tipo VENTA y que sean del departamento de JOYERÍA.

## 4. Genere una consulta que cree un campo “co_ranking” que clasifique en orden ascendente las campañas según el total de venta, siendo 1 la campaña que más ha vendido y N la que menos. (N = Total de campañas)

In [13]:
sql_query("""
        SELECT c.id_campania, sum(v.precio) as total_vendido, ROW_NUMBER() OVER (ORDER BY sum(v.precio) DESC) as co_ranking

          FROM campania c

          LEFT JOIN DEPTO_CAMPANIA dp
          ON c.id_campania=dp.id_campania

          LEFT JOIN ARTICULO a
          ON dp.id_dpto=a.id_dpto

          LEFT JOIN VENTA v
          ON a.referencia=v.referencia

          GROUP BY c.id_campania
          ;""")

,ID_CAMPANIA,total_vendido,co_ranking
0,CA2,360.84,1
1,CA1,93.99,2
2,CA4,47.00,3
3,CA3,46.99,4
4,CA6,46.99,5
5,CA5,NaN,6


## 5. Clasifique todas las ventas en Mayor, Igual o Menor precio respecto a la media de los precios de todas las ventas realizadas.

**Resultado**:


![tablares](../../../img/Tablas-4.png)

In [5]:
sql_query("""SELECT talon, precio, 
        CASE WHEN precio > (SELECT AVG(precio) FROM venta) THEN 'Mayor' ELSE 'Menor' END AS Clasificacion
        FROM venta
          ;""")

,TALON,PRECIO,Clasificacion
0,01,15.00,Menor
1,02,18.00,Menor
2,03,14.00,Menor
3,04,37.95,Menor
4,05,125.90,Mayor
5,06,150.00,Mayor
6,07,22.99,Menor
7,08,24.00,Menor


# 7. Genere una consulta que para cada campaña de tipo “venta” obtenga el talón medio, precio medio, nº de referencias compradas y nº de talones con venta superior al talón medio.

## 8. Genere una o varias consultas que devulevan las combinaciones de familias y campañas cuya venta haya sido superior a 100€.

In [6]:
sql_query("""SELECT f.cod_familia, dc.id_campania,SUM(v.precio) as total
            FROM familia f
            LEFT JOIN depto_campania dc
            ON f.id_dpto=dc.id_dpto
            LEFT JOIN CAMPANIA c
            ON dc.id_campania=c.id_campania
            LEFT JOIN articulo a
            ON dc.id_dpto=a.id_dpto
            LEFT JOIN venta v
            ON a.referencia=v.referencia
            GROUP BY f.cod_familia, dc.id_campania
            HAVING SUM(v.precio>100)
          ;""")

,COD_FAMILIA,ID_CAMPANIA,total
0,2F1,CA2,313.85


## 9. Genere una o varias consultas que permitan catalogar los artículos vendidos y que no son de la Campaña CA1, según si se tratan de artículos de Ropa o Accesorios. En base a los departamentos y familias. NOTA: Se valorará la elaboración de la consulta y no la veracidad del propio catálogo)

## 10. Genere una o varias consultas que devuelvan la variación del total de venta entre años. Tomando como variación la siguiente fórmula $$\dfrac{Year_{n} – Year_{n-1}}{Year_{n-1}}$$ (Se valorará el control de valores 0 en la división o de indeterminaciones)

![tablas5](../../../img/Tablas-5.png)

In [57]:
sql_query("""SELECT anio AS anio_1, 
       LAG(anio) OVER (ORDER BY anio DESC) AS anio_2, 
       (LAG(total_anio) OVER (ORDER BY anio DESC) - total_anio) /total_anio  AS variacion
       
FROM (SELECT SUBSTR(fecha_venta, -4) AS anio, SUM(precio) as total_anio
FROM venta
GROUP BY anio
ORDER BY anio DESC)
ORDER BY anio DESC;""")

,anio_1,anio_2,variacion
0,2013,None,NaN
1,2012,2013,-0.575758
2,2011,2012,-0.832479
3,2010,2011,0.202258
